In [23]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

# #sklearnex
# from sklearnex import patch_sklearn
# patch_sklearn()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
import cv2 as cv
from radiomics import featureextractor
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.preprocessing import StandardScaler

#Import paths and patients classes
from notebooks.info import path_label, patient
import notebooks.utils as utils

# Feature extraction

## Functions

In [2]:
def extractor_settings(param_path, show=False):
    """set extraction settings for pyradiomics

    Args:
        param_path (str): relative path of parameter file
        show (bool, optional): if printing setting or not. Defaults to False.

    Returns:
        obj: extractor of pyradiomics
    """
    extractor = featureextractor.RadiomicsFeatureExtractor(str(repo_path /param_path))
    if show:
        print('Extraction parameters:\n\t', extractor.settings)
        print('Enabled filters:\n\t', extractor.enabledImagetypes)
        print('Enabled features:\n\t', extractor.enabledFeatures)
    return extractor

In [3]:
def feature_extraction(df: pd.DataFrame, pat: object, rad: str, time: int, stype: str, i: int):
    im = sitk.JoinSeries(pat.im_sitk('SET')) #Add dimension to be able to use pyradiomics. Image (a,b) turns into (a,b,1)
    #get radiomics features
    param_path = 'data/param_files/Param_64bin_all_radiomics.json' #path of parameter file
    extractor = extractor_settings(param_path, show=False)
    #mask path
    mask_path = str(repo_path / pat.seg_path(rad, time, stype)[0])
    #extract
    result = extractor.execute(im,mask_path) # Extract features
    #feature vector length
    fv_len = 102
    print(list(result.values())[-fv_len:])
    #create df is not yet defined
    if df is None:
        column_names = list(result.keys())[-fv_len:] #get column names
        column_names = [x.replace('original_','') for x in column_names] #remove original_ string
        column_names.insert(0, 'pat_num') #insert pat_num at the beginning
        df = pd.DataFrame(columns=column_names)
    #add feature vector to df if it has values
    feature_vector = list(result.values())[-fv_len:] #get feature vector
    feature_vector.insert(0, pat.pat_num) #insert pat_num at the beginning
    #add feature vector according to loc
    df.loc[i] = feature_vector
    return df

## Implementation

In [31]:
info = path_label()

# segmentation type
stype='G'
general_df = None
for rad in ['L', 'V', 'M']:
    for time in [1,2]:
        df = None # df to store all feature vectors
        for i in tqdm(range(info.len)): # go thourgh all patients
            pat = patient(info, num=i)
            df = feature_extraction(df, pat, rad, time, stype, i)
            clear_output(wait=True)
        general_df = pd.concat([general_df, df], ignore_index=True)
        df.to_csv(repo_path / 'data' / 'features' / f'features_{rad}_{time}_{stype}.csv')
general_df.to_csv(repo_path / 'data' / 'features' / f'features_all_time{stype}.csv')

100%|██████████| 33/33 [00:09<00:00,  3.39it/s]


# Predictions

In [5]:
def get_ex_included(budget_path: Path):
    # get the name of the features from the budget
    budget = pd.read_excel( budget_path, index_col=0)
    # change name of column
    budget.columns = ['budget']
    # get all features with values greater than 1
    excluded = budget[budget[ 'budget' ] > 1].index
    # get all other names
    included = budget[budget[ 'budget' ] <= 1].index
    return excluded, included

def get_features(stype:str, excluded:list):
    # get features
    features = pd.read_csv(repo_path / 'data' / 'features' / f'features_all_time{stype}.csv', index_col=0)
    features = features.groupby(by='pat_num', axis=0).mean()
    # remove features in excluded list
    features = features.drop(excluded, axis=1)

    return features

## Load data

In [6]:
# HP
stype='G' # segmentation type
label = 'RP' # receptor type (RP, RE, ki67
budget_path = repo_path/ 'data' / 'budget' / 'budget.xlsx'

excluded, _ = get_ex_included(budget_path) # get excluded features due to their budget value
features = get_features(stype, excluded) 
info = path_label()

## LASSO for feature selection

In [67]:
num_features = 10

# use lasso to select features
lasso = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
lasso.fit(features, info.labels_list(label))

# use lasso to select the 5 features with the largest coefficients
coef = pd.Series(lasso.coef_[0], index=features.columns)
coef = coef.sort_values(ascending=False)
coef = coef[coef != 0]
coef = coef[:num_features]
print(f'The {num_features} features with the largest coefficients are:\n{coef}\n')

The 10 features with the largest coefficients are:
firstorder_InterquartileRange                0.008038
firstorder_RobustMeanAbsoluteDeviation       0.007211
firstorder_MeanAbsoluteDeviation             0.006907
gldm_SmallDependenceHighGrayLevelEmphasis    0.004517
glcm_SumAverage                              0.002541
glszm_SmallAreaHighGrayLevelEmphasis         0.001575
firstorder_Range                             0.001543
firstorder_Variance                          0.001179
glrlm_ShortRunHighGrayLevelEmphasis          0.001065
glcm_Autocorrelation                         0.001062
dtype: float64



## Train

### Logistic regression

In [68]:
x = features[coef.index]
# scale features using standard scaler
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = info.labels_list(label)

logreg = LogisticRegression(max_iter=1000)
logreg.fit(x, y)
# get the accuracy of the model
print(f'Accuracy: {logreg.score(x, y)}')
print(f'A dumb prediction would have an accuracy of: {np.mean(y)}')

Accuracy: 0.8181818181818182
A dumb prediction would have an accuracy of: 0.6363636363636364


### Random forest

In [74]:
# run a lasso classifier with all the features using leave one out cross validation
# LOOCV
loo = LeaveOneOut()
loo.get_n_splits(features)
scores = []
y = np.array(y)
for train_index, test_index in loo.split(features):
    print(f'{i+1}')
    X_train, X_test = features.iloc[train_index][coef.index], features.iloc[test_index][coef.index]
    y_train, y_test = y[train_index], y[test_index]
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    clf = LogisticRegression(penalty='l2', max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))
print(f'Accuracy: {np.mean(scores)}')

NameError: name 'i' is not defined